In [15]:
import websocket
import json
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta
import sympy as sp
import scipy as sc
from time import sleep
from binance import Client
# client = Client(api_key="jM9zEeTTzT7R7QGlNrh5tUpOc8l71hOxu7CewCL4M0NvbBp3ilgmEwMOOjJfCRVB",api_secret="iZxFmlz5bvROsHQPZTP5hlJtsxYjOu8wocOlLWzIDdhMzEQu7KyyHB2DseOYQ66s",testnet=True) # Tinmar Testnet
# client = Client(api_key="VQaW84v81fvM6b0z2cG97q0FBGK0VDVNYZGxPlMYkL1caao8u7lSrSWxeE5c6NKq",api_secret="9XgnO5nds9V8UVw5O8D0J6NHNPqL4ojj5t3FxZs7xqc8sd8MwEt44QshkT3W9auM",tld='com') # Tinmar Real
client = Client(api_key="d9yKX3gEa9Hn0jOiVKTte8B55VRmwmnG4QYbpXO59UEWDyKSKf1XcyoYX2SYBwT8",api_secret="PLCQtT9PeDmZMenRmdUj4ONoPDv5Ph1xHdO8IFOCSK2BXfNXXts9JfFU56MKL8fz",tld='com') # Cristian Real


# from google.cloud.sql.connector import Connector
# from sqlalchemy import create_engine, text
# import pg8000.native

# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/tinma/OneDrive/Escritorio/application_default_credentials.json'

# connector = Connector()

# def getconn() -> pg8000.native.Connection:
#     conn: pg8000.native.Connection = connector.connect(
#         "semiotic-pact-388619:us-central1:tradingbot",
#         "pg8000",
#         user="postgres",
#         password="GuruSat.3",
#         db="criptos"
#     )
#     return conn

# connection = create_engine('postgresql+pg8000://',creator=getconn)

In [17]:
client.get_account(recvWindow=60000)

{'makerCommission': 10,
 'takerCommission': 10,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00100000',
  'taker': '0.00100000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1712967279737,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BTC', 'free': '0.00005891', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '0.00001532', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'EOS', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'SNT', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNT', 'free': '0.00000000', 'locke

In [4]:
datetime.utcnow()

datetime.datetime(2024, 4, 9, 10, 34, 10, 247067)

In [111]:
import websocket
import json
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta
import sympy as sp
import scipy as sc
from time import sleep
from binance import Client
# client = Client(api_key="jM9zEeTTzT7R7QGlNrh5tUpOc8l71hOxu7CewCL4M0NvbBp3ilgmEwMOOjJfCRVB",api_secret="iZxFmlz5bvROsHQPZTP5hlJtsxYjOu8wocOlLWzIDdhMzEQu7KyyHB2DseOYQ66s",testnet=True)
client = Client(api_key="d9yKX3gEa9Hn0jOiVKTte8B55VRmwmnG4QYbpXO59UEWDyKSKf1XcyoYX2SYBwT8",api_secret="PLCQtT9PeDmZMenRmdUj4ONoPDv5Ph1xHdO8IFOCSK2BXfNXXts9JfFU56MKL8fz",tld='com') # Cristian Real


############################################################################################################################################################
# INICIO DEL BOT DE TRADING.
############################################################################################################################################################

##################################################
# PARÁMETROS INICIALES.
##################################################
###################################
# Parámetros de interpolación.
###################################
smooth_interval = 30
smooth_exp = 3

###################################
# Parámetros de estrategia.
###################################
rango = 5
std_mult = 40
pdi_mult = 10
sdi_mult = 100
z_aprox = 0.01
pmax_idx = 0
pmin_idx = 0
last_time_buy = datetime.utcnow()
last_time_sell = datetime.utcnow()

###################################
# Metadatos de la estrategia.
###################################
estrategia = 'DERIVATIVES_TESTNET_V1'
trade_qty = 0.00075
k = 0
bullish = False
bearish = False

###################################
# Data Frames y Listas de Trabajo.
###################################
buy_orders = []
sell_orders = []
max_points_idx = []
min_points_idx = []
df_logs = pd.DataFrame(columns=['trade_id','orden_id','client_order_id','cantidad','cantidad_ejecutada','accion','tiempo','precio','comision','activo_comision'])

############################################################################################################################################################
# Ciclo anterior al streaming para obtener mejores LOGS y menos código innecesario.
#############################################################################################################################################################
while True:
    
    #############################################
    # Limpieza y almacenamiento de los datos de streaming en el df correspondiente.
    #############################################
    recent_data = pd.read_csv('../../../csvs/streaming/BTCUSDT.csv')
    recent_data.columns = ['Tiempo','Simbolo','Precio']
    recent_data = recent_data.loc[:,['Tiempo','Precio']]
    recent_data['Tiempo'] = recent_data.Tiempo.astype('datetime64')
    recent_data = recent_data.set_index('Tiempo')
    recent_data['Precio'] = recent_data.Precio.astype(float)
    recent_data = recent_data.reset_index()
    
    if k == 0:
        print(f'Inicia el bot de trading con inicio en: {datetime.utcnow()}')
        k += 1

    ##################################################################################################################################
    # INICIA BOT DE TRADING: ACTUALIZACIÓN DE DATOS, CÁLCULO DE DERIVADAS, ESTRATEGIA Y TRADES AUTOMÁTICOS.
    ##################################################################################################################################    
    ##################################################################################################################################
    # CÁLCULO DE DERIVADAS, ESTRATEGIA Y TRADES AUTOMÁTICOS.
    ##################################################################################################################################        
    #############################################
    # CÁLCULO DE DERIVADAS
    #############################################
    recent_data['Interpolacion'] = sc.signal.savgol_filter(recent_data.Precio,smooth_interval,smooth_exp)
    recent_data['Primer_Derivada'] = recent_data.Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
    recent_data['PD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Primer_Derivada,smooth_interval,smooth_exp)
    recent_data['Segunda_Derivada'] = recent_data.PD_Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
    recent_data['SD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Segunda_Derivada,smooth_interval,smooth_exp)
    recent_data['Tercer_Derivada'] = recent_data.SD_Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
    recent_data['TD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Tercer_Derivada,smooth_interval,3)
    # recent_data = recent_data.dropna()

    ######################################################################
    # Agregamos el código necesario para encontrar los puntos exactos mínimos y máximos
    ######################################################################
    try:
        initial_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo.iloc[-60*15:-60*15 + 1].item()),'%Y-%m-%d %H:%M:%S.%f')].index.item() + rango
    except:
        initial_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo.iloc[-60*15:-60*15 + 1].item()),'%Y-%m-%d %H:%M:%S')].index.item() + rango
    try:
        final_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo.iloc[-1:].item()),'%Y-%m-%d %H:%M:%S.%f')].index.item() - rango
    except:
        final_date_index = recent_data[recent_data.Tiempo == datetime.strptime(str(recent_data.Tiempo.iloc[-1:].item()),'%Y-%m-%d %H:%M:%S')].index.item() - rango

    lista_min_max = recent_data.iloc[initial_date_index:final_date_index][(recent_data.iloc[initial_date_index:final_date_index]['TD_Interpolacion'] >= -z_aprox) & (recent_data.iloc[initial_date_index:final_date_index]['TD_Interpolacion'] <= z_aprox)]
    indices_min_max = lista_min_max.index.tolist()
    mp = recent_data.Precio.iloc[initial_date_index:final_date_index].mean()

    pmin = 0
    pmax = 0

    #############################################
    # ESTRATEGIA
    #############################################
    
    for PMSD in indices_min_max:
        if (recent_data.SD_Interpolacion[PMSD:PMSD+1].item() > 0) & (recent_data.PD_Interpolacion[PMSD:PMSD+1].item() > 0):
            if (abs(recent_data.SD_Interpolacion[PMSD]*10) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)) & ((recent_data.PD_Interpolacion[PMSD]*pdi_mult) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)):
                pmax = recent_data.SD_Interpolacion[PMSD]
            for i in range(rango+1):
                if (i > PMSD) or (PMSD+i >= indices_min_max[-1:][0]):
                    break
                elif ((recent_data.SD_Interpolacion[PMSD-i] > pmax)) & ((abs(recent_data.SD_Interpolacion[PMSD-i]*sdi_mult)) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)) & ((recent_data.PD_Interpolacion[PMSD-i]*pdi_mult) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)):
                    pmax_idx = PMSD-i
                elif ((recent_data.SD_Interpolacion[PMSD+i] > pmax)) & ((abs(recent_data.SD_Interpolacion[PMSD+i]*sdi_mult)) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)) & ((recent_data.PD_Interpolacion[PMSD+i]*pdi_mult) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)):
                    pmax_idx = PMSD+i
            max_points_idx.append(pmax_idx)
        elif (recent_data.SD_Interpolacion[PMSD:PMSD+1].item()) < 0 & (recent_data.PD_Interpolacion[PMSD:PMSD+1].item() < 0):
            if (abs(recent_data.SD_Interpolacion[PMSD]*10) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)) & ((recent_data.PD_Interpolacion[PMSD]*pdi_mult) < -abs(np.std(recent_data.SD_Interpolacion)*std_mult)):
                pmin = recent_data.SD_Interpolacion[PMSD]
            for i in range(rango+1):
                if (i > PMSD) or (PMSD+i >= indices_min_max[-1:][0]):
                    break
                elif ((recent_data.SD_Interpolacion[PMSD-i] < pmin)) & (abs(recent_data.SD_Interpolacion[PMSD-i]*sdi_mult) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)) & ((recent_data.PD_Interpolacion[PMSD-i]*pdi_mult) < -abs(np.std(recent_data.SD_Interpolacion)*std_mult)):
                    pmin_idx = PMSD-i
                elif ((recent_data.SD_Interpolacion[PMSD+i] < pmin)) & (abs(recent_data.SD_Interpolacion[PMSD+i]*sdi_mult) > abs(np.std(recent_data.SD_Interpolacion)*std_mult)) & ((recent_data.PD_Interpolacion[PMSD+i]*pdi_mult) < -abs(np.std(recent_data.SD_Interpolacion)*std_mult)):
                    pmin_idx = PMSD+i
            min_points_idx.append(pmin_idx)
    max_points_idx = list(set(max_points_idx))
    min_points_idx = list(set(min_points_idx))
    max_points_idx.sort()
    min_points_idx.sort()
    
    #############################################
    # AUTOMATIC TRADING.
    #############################################
    # try:
    if bullish == False:
        if (recent_data.Tiempo[max_points_idx[-1]] != last_time_buy) & (recent_data.Tiempo[max_points_idx[-1]] > last_time_buy) & (recent_data.Tiempo[max_points_idx[-1]] > last_time_sell):
            last_time_buy = recent_data.Tiempo[max_points_idx[-1]]
            # print(f'Precio de compra en: {recent_data.Precio[max_points_idx[-1]]}. Tiempo de compra: {recent_data.Tiempo[max_points_idx[-1]]}. Tiempo real de la señal: {datetime.utcnow()}. Diferencia entre señal y compra: {(datetime.utcnow() - recent_data.Tiempo[max_points_idx[-1]]).seconds} segundos.')
            marketorder_buy = client.create_order(
                symbol='BTCFDUSD',
                side=Client.SIDE_BUY,
                type=Client.ORDER_TYPE_MARKET,
                quantity=trade_qty,
                recvWindow=60000)
            buy_orders.append(marketorder_buy)
            
            entry_price = float(marketorder_buy['fills'][0]['price'])
            entry_time = datetime.fromtimestamp(marketorder_buy['transactTime']/1000)
            TID = marketorder_buy['fills'][0]['tradeId']
            OID = marketorder_buy['orderId'] # orderId
            COID = marketorder_buy['clientOrderId'] # clientOrderId
            Qty = marketorder_buy['origQty'] # origQty
            exc_Qty = marketorder_buy['executedQty']
            side = marketorder_buy['side'] # Side
            commission = marketorder_buy['fills'][0]['commission'] # Commission
            commissionAsset = marketorder_buy['fills'][0]['commissionAsset'] # commissionAsset

            lista_logs = [TID,OID,COID,Qty,exc_Qty,side,entry_time,entry_price,commission,commissionAsset]
            df_logs.loc[len(df_logs)] = lista_logs
            
            bullish = True
            # bearish = False
            print(f'TIEMPO SEÑAL COMPRA: Precio; {recent_data.Precio[max_points_idx[-1]]}. Tiempo; {recent_data.Tiempo[max_points_idx[-1]]}. TIEMPO COMPRA: Precio; {entry_price}. Tiempo; {entry_time}.')
            print(f'DIFERENCIA ENTRE SEÑAL Y COMPRA: {(datetime.utcnow() - recent_data.Tiempo[max_points_idx[-1]]).seconds} segundos.')            
    elif bullish == True:
        if (recent_data.Tiempo[min_points_idx[-1]] != last_time_sell) & (recent_data.Tiempo[min_points_idx[-1]] > last_time_sell) & (recent_data.Tiempo[min_points_idx[-1]] > last_time_buy):
            last_time_sell = recent_data.Tiempo[min_points_idx[-1]]
            # print(f'Precio de venta en: {recent_data.Precio[min_points_idx[-1]]}. Tiempo de venta: {recent_data.Tiempo[min_points_idx[-1]]}. Tiempo real de la señal: {datetime.utcnow()}. Diferencia entre señal y compra: {(datetime.utcnow() - recent_data.Tiempo[min_points_idx[-1]]).seconds} segundos.')
            marketorder_sell = client.create_order(
                symbol='BTCFDUSD',
                side=Client.SIDE_SELL,
                type=Client.ORDER_TYPE_MARKET,
                quantity=trade_qty,
                recvWindow=60000)
            sell_orders.append(marketorder_sell)
            
            exit_price = float(marketorder_sell['fills'][0]['price'])
            exit_time = datetime.fromtimestamp(marketorder_sell['transactTime']/1000)
            TID = marketorder_sell['fills'][0]['tradeId']
            OID = marketorder_sell['orderId'] # orderId
            COID = marketorder_sell['clientOrderId'] # clientOrderId
            Qty = marketorder_sell['origQty'] # origQty
            exc_Qty = marketorder_sell['executedQty']
            side = marketorder_sell['side'] # Side
            commission = marketorder_sell['fills'][0]['commission'] # Commission
            commissionAsset = marketorder_sell['fills'][0]['commissionAsset'] # commissionAsset
            
            lista_logs = [TID,OID,COID,Qty,exc_Qty,side,exit_time,exit_price,commission,commissionAsset]
            df_logs.loc[len(df_logs)] = lista_logs
            
            bullish = False
            # bearish = True
            print(f'TIEMPO SEÑAL VENTA: Precio; {recent_data.Precio[min_points_idx[-1]]}. Tiempo; {recent_data.Tiempo[min_points_idx[-1]]}. TIEMPO VENTA: Precio: {exit_price}. Tiempo: {exit_time}. RPNL: {(exit_price-entry_price)*trade_qty} FDUSD')
            print(f'DIFERENCIA ENTRE SEÑAL Y VENTA: {(datetime.utcnow() - recent_data.Tiempo[min_points_idx[-1]]).seconds} segundos.')    
        # except:
    #     pass
    
    if recent_data.Tiempo.iloc[-1:].item() >= datetime.strptime('2024-04-13 21:00:00','%Y-%m-%d %H:%M:%S'):
        break

Inicia el bot de trading con inicio en: 2024-04-13 06:22:30.104271
TIEMPO SEÑAL COMPRA: Precio; 67318.69. Tiempo; 2024-04-13 06:23:37.635000. TIEMPO COMPRA: Precio; 67143.91. Tiempo; 2024-04-13 00:23:54.179000.
DIFERENCIA ENTRE SEÑAL Y COMPRA: 17 segundos.


In [103]:
marketorder_buy

{'symbol': 'BTCFDUSD',
 'orderId': 4948334298,
 'orderListId': -1,
 'clientOrderId': 'XsVPMBUwW5q0GbRJsOxRNM',
 'transactTime': 1712978910555,
 'price': '0.00000000',
 'origQty': '0.00025000',
 'executedQty': '0.00025000',
 'cummulativeQuoteQty': '16.64622000',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'BUY',
 'workingTime': 1712978910555,
 'fills': [{'price': '66584.88000000',
   'qty': '0.00025000',
   'commission': '0.00000000',
   'commissionAsset': 'BNB',
   'tradeId': 592745686}],
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [79]:
df_logs

,Tiempo,Accion
0,2024-04-13 00:26:15.589018,C
1,2024-04-13 00:27:42.681150,V
2,2024-04-13 00:29:43.675752,C
3,2024-04-13 00:30:31.240674,V
4,2024-04-13 00:30:51.510956,C
...,...,...
56,2024-04-13 01:23:00.160157,C
57,2024-04-13 01:23:01.650074,V
58,2024-04-13 01:26:06.999152,C
59,2024-04-13 01:26:47.399518,V


In [88]:
orders[orders.origQty == 0.00025].time

9    2024-04-13 00:26:15.319
10   2024-04-13 00:27:42.410
11   2024-04-13 00:29:43.405
12   2024-04-13 00:30:30.967
13   2024-04-13 00:30:51.237
               ...          
66   2024-04-13 01:23:01.368
67   2024-04-13 01:26:06.716
68   2024-04-13 01:26:47.116
69   2024-04-13 01:27:13.765
70   2024-04-13 01:31:13.226
Name: time, Length: 62, dtype: datetime64[ns]

In [81]:
orders = client.get_all_orders(symbol='BTCTUSD')
orders = pd.DataFrame(orders)
orders

,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,workingTime,origQuoteOrderQty,selfTradePreventionMode
0,BTCTUSD,3642046837,-1,and_84dc04c1d1d84fad8c86a7b5b7f485ed,0.00000000,0.00014000,0.00014000,9.39142960,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1712962352177,1712962352177,True,1712962352177,10.00000000,EXPIRE_MAKER
1,BTCTUSD,3642049567,-1,and_906a064b9618469c8a83f4feb80ae8f9,0.00000000,0.00085000,0.00085000,57.00400190,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1712962538514,1712962538514,True,1712962538514,57.60857040,EXPIRE_MAKER
2,BTCTUSD,3642058873,-1,and_f994cbe299e5442da5c5d06c6fe2986b,0.00000000,0.00099000,0.00099000,66.28255350,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,1712963109701,1712963109701,True,1712963109701,0.00000000,EXPIRE_MAKER
3,BTCTUSD,3642068784,-1,and_48a7991553bf466a8df18138a120796d,0.00000000,0.00099000,0.00099000,66.35086260,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1712963666894,1712963666894,True,1712963666894,66.82415350,EXPIRE_MAKER
4,BTCTUSD,3642082865,-1,and_2bd83a8e16d54525b671d10ce5065507,0.00000000,0.00099000,0.00099000,66.34127610,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,1712964384211,1712964384211,True,1712964384211,0.00000000,EXPIRE_MAKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,BTCTUSD,3642233859,-1,677cfTyvz8cKGyrAmb9Gko,0.00000000,0.00025000,0.00025000,16.64389000,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,1712971381368,1712971381368,True,1712971381368,0.00000000,EXPIRE_MAKER
67,BTCTUSD,3642237349,-1,pvbDXHiqTlqYyaAdQyH6JK,0.00000000,0.00025000,0.00025000,16.64180500,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1712971566716,1712971566716,True,1712971566716,0.00000000,EXPIRE_MAKER
68,BTCTUSD,3642238226,-1,Q5rmwwcuuNdf4A3SZaZ65z,0.00000000,0.00025000,0.00025000,16.63885750,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,1712971607116,1712971607116,True,1712971607116,0.00000000,EXPIRE_MAKER
69,BTCTUSD,3642238968,-1,sVE7LxPYCQKpPjHCHaYGvM,0.00000000,0.00025000,0.00025000,16.64724750,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1712971633765,1712971633765,True,1712971633765,0.00000000,EXPIRE_MAKER


In [82]:
orders.time = pd.to_datetime(orders.time,unit='ms')
orders

,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,workingTime,origQuoteOrderQty,selfTradePreventionMode
0,BTCTUSD,3642046837,-1,and_84dc04c1d1d84fad8c86a7b5b7f485ed,0.00000000,0.00014000,0.00014000,9.39142960,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2024-04-12 22:52:32.177,1712962352177,True,1712962352177,10.00000000,EXPIRE_MAKER
1,BTCTUSD,3642049567,-1,and_906a064b9618469c8a83f4feb80ae8f9,0.00000000,0.00085000,0.00085000,57.00400190,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2024-04-12 22:55:38.514,1712962538514,True,1712962538514,57.60857040,EXPIRE_MAKER
2,BTCTUSD,3642058873,-1,and_f994cbe299e5442da5c5d06c6fe2986b,0.00000000,0.00099000,0.00099000,66.28255350,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,2024-04-12 23:05:09.701,1712963109701,True,1712963109701,0.00000000,EXPIRE_MAKER
3,BTCTUSD,3642068784,-1,and_48a7991553bf466a8df18138a120796d,0.00000000,0.00099000,0.00099000,66.35086260,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2024-04-12 23:14:26.894,1712963666894,True,1712963666894,66.82415350,EXPIRE_MAKER
4,BTCTUSD,3642082865,-1,and_2bd83a8e16d54525b671d10ce5065507,0.00000000,0.00099000,0.00099000,66.34127610,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,2024-04-12 23:26:24.211,1712964384211,True,1712964384211,0.00000000,EXPIRE_MAKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,BTCTUSD,3642233859,-1,677cfTyvz8cKGyrAmb9Gko,0.00000000,0.00025000,0.00025000,16.64389000,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,2024-04-13 01:23:01.368,1712971381368,True,1712971381368,0.00000000,EXPIRE_MAKER
67,BTCTUSD,3642237349,-1,pvbDXHiqTlqYyaAdQyH6JK,0.00000000,0.00025000,0.00025000,16.64180500,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2024-04-13 01:26:06.716,1712971566716,True,1712971566716,0.00000000,EXPIRE_MAKER
68,BTCTUSD,3642238226,-1,Q5rmwwcuuNdf4A3SZaZ65z,0.00000000,0.00025000,0.00025000,16.63885750,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,2024-04-13 01:26:47.116,1712971607116,True,1712971607116,0.00000000,EXPIRE_MAKER
69,BTCTUSD,3642238968,-1,sVE7LxPYCQKpPjHCHaYGvM,0.00000000,0.00025000,0.00025000,16.64724750,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2024-04-13 01:27:13.765,1712971633765,True,1712971633765,0.00000000,EXPIRE_MAKER


In [48]:
recent_data[recent_data.Tiempo > df_logs.Tiempo[0]].head(1).index

Int64Index([11543], dtype='int64')

In [49]:
recent_data.Tiempo[11543]

Timestamp('2024-04-13 00:26:16.394000')

In [35]:
recent_data

,Tiempo,Precio,Interpolacion,Primer_Derivada,PD_Interpolacion,Segunda_Derivada,SD_Interpolacion,Tercer_Derivada,TD_Interpolacion
0,2024-04-12 20:57:23.315,66952.01,66954.967836,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-04-12 20:57:24.370,66944.89,66950.591200,-4.148470,NaN,NaN,NaN,NaN,NaN
2,2024-04-12 20:57:25.401,66944.90,66946.832540,-3.645645,NaN,NaN,NaN,NaN,NaN
3,2024-04-12 20:57:26.160,66944.89,66943.674630,-4.160619,NaN,NaN,NaN,NaN,NaN
4,2024-04-12 20:57:27.351,66944.90,66941.100245,-2.161532,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
15351,2024-04-13 01:29:57.292,66561.12,66561.123305,0.000359,0.000340,0.000169,0.000194,0.000028,-0.000035
15352,2024-04-13 01:29:58.142,66561.12,66561.123896,0.000695,0.000561,0.000260,0.000227,0.000039,-0.000004
15353,2024-04-13 01:29:59.286,66561.12,66561.124690,0.000695,0.000815,0.000222,0.000262,0.000031,0.000040
15354,2024-04-13 01:30:00.083,66561.13,66561.125707,0.001275,0.001104,0.000363,0.000300,0.000047,0.000099


In [52]:
pd.concat([pmax_df,pmin_df],axis=0).sort_values('Tiempo')

,Tiempo,Precio,Accion
11543,2024-04-13 00:26:16.394,66924.00,C
11543,2024-04-13 00:26:16.394,66924.00,V
11750,2024-04-13 00:29:44.145,66987.91,C
11750,2024-04-13 00:29:44.145,66987.91,V
11818,2024-04-13 00:30:52.609,67090.89,C
...,...,...,...
14936,2024-04-13 01:23:00.885,66484.06,V
14936,2024-04-13 01:23:00.885,66484.06,C
15122,2024-04-13 01:26:07.346,66488.00,V
15122,2024-04-13 01:26:07.346,66488.00,C


In [77]:
lidx_buy = []
lidx_sell = []

for i in range(len(df_logs)):
    if df_logs.Accion[i] == 'C':
        idx_buy = recent_data[recent_data.Tiempo > df_logs.Tiempo[i]].head(1).index.item()
        lidx_buy.append(idx_buy)
    elif df_logs.Accion[i] == 'V':
        idx_sell = recent_data[recent_data.Tiempo > df_logs.Tiempo[i]].head(1).index.item()
        lidx_sell.append(idx_sell)

pmax_df = recent_data.iloc[lidx_buy].loc[:,['Tiempo','Precio']]
pmax_df['Accion'] = 'C'
pmin_df = recent_data.iloc[lidx_sell].loc[:,['Tiempo','Precio']]
pmin_df['Accion'] = 'V'
vc_df = pd.concat([pmax_df,pmin_df],axis=0)
vc_df = vc_df.reset_index()
vc_df = vc_df.set_index('index')
vc_df = vc_df.sort_index()
del_idx = []
vc_df = vc_df.reset_index()
for i in vc_df.index:
    try:
        if vc_df.Accion[i+1] == vc_df.Accion[i]:
            del_idx.append(i+1)
    except:
        pass
vc_df.drop(del_idx,inplace=True)
vc_df = vc_df.reset_index(drop=True)
vc_df['Diferencia_Precio_USDT'] = pd.Series(dtype='float')
for i in range(len(vc_df)-1):
    vc_df['Diferencia_Precio_USDT'][i+1] = (vc_df.Precio[i+1] - vc_df.Precio[i]) * 0.0001
profits = 0
for i in range(len(vc_df)-1):
    if vc_df.Accion[i] == 'C' and vc_df.Diferencia_Precio_USDT[i+1] > 0:
        profits += vc_df.Diferencia_Precio_USDT[i+1]
    elif vc_df.Accion[i] == 'C' and vc_df.Diferencia_Precio_USDT[i+1] < 0:
        profits -= vc_df.Diferencia_Precio_USDT[i+1]
    elif vc_df.Accion[i] == 'V' and vc_df.Diferencia_Precio_USDT[i+1] > 0:
        profits += vc_df.Diferencia_Precio_USDT[i+1]
    elif vc_df.Accion[i] == 'V' and vc_df.Diferencia_Precio_USDT[i+1] < 0:
        profits -= vc_df.Diferencia_Precio_USDT[i+1]
profits

C:\Users\tinma\AppData\Local\Temp\ipykernel_28576\836296614.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vc_df['Diferencia_Precio_USDT'][i+1] = (vc_df.Precio[i+1] - vc_df.Precio[i]) * 0.0001


0.1971640000000043

In [98]:
62.64+3.94-0.8348064599999955


65.74519354

In [97]:
0.1971640000000043 - (0.01663886*62)

-0.8344453199999955

In [78]:
vc_df

,index,Tiempo,Precio,Accion,Diferencia_Precio_USDT
0,11543,2024-04-13 00:26:16.394,66924.00,C,NaN
1,11630,2024-04-13 00:27:42.859,66936.38,V,0.001238
2,11750,2024-04-13 00:29:44.145,66987.91,C,0.005153
3,11798,2024-04-13 00:30:32.022,67028.02,V,0.004011
4,11818,2024-04-13 00:30:52.609,67090.89,C,0.006287
...,...,...,...,...,...
56,14936,2024-04-13 01:23:00.885,66484.06,C,-0.011196
57,14937,2024-04-13 01:23:01.879,66502.74,V,0.001868
58,15122,2024-04-13 01:26:07.346,66488.00,C,-0.001474
59,15162,2024-04-13 01:26:47.487,66478.84,V,-0.000916


In [75]:
orders = client.get_all_orders(symbol='BTCTUSD')
orders

[{'symbol': 'BTCTUSD',
  'orderId': 3642046837,
  'orderListId': -1,
  'clientOrderId': 'and_84dc04c1d1d84fad8c86a7b5b7f485ed',
  'price': '0.00000000',
  'origQty': '0.00014000',
  'executedQty': '0.00014000',
  'cummulativeQuoteQty': '9.39142960',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'BUY',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1712962352177,
  'updateTime': 1712962352177,
  'isWorking': True,
  'workingTime': 1712962352177,
  'origQuoteOrderQty': '10.00000000',
  'selfTradePreventionMode': 'EXPIRE_MAKER'},
 {'symbol': 'BTCTUSD',
  'orderId': 3642049567,
  'orderListId': -1,
  'clientOrderId': 'and_906a064b9618469c8a83f4feb80ae8f9',
  'price': '0.00000000',
  'origQty': '0.00085000',
  'executedQty': '0.00085000',
  'cummulativeQuoteQty': '57.00400190',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'BUY',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 171296253851

In [76]:
marketorder_sell

# orderId, clientOrderId, transactTime, origQty, executedQty, side, fills: price, commission, commissionAsset, tradeId

{'symbol': 'BTCTUSD',
 'orderId': 3642238226,
 'orderListId': -1,
 'clientOrderId': 'Q5rmwwcuuNdf4A3SZaZ65z',
 'transactTime': 1712971607116,
 'price': '0.00000000',
 'origQty': '0.00025000',
 'executedQty': '0.00025000',
 'cummulativeQuoteQty': '16.63885750',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'workingTime': 1712971607116,
 'fills': [{'price': '66555.43000000',
   'qty': '0.00025000',
   'commission': '0.01663886',
   'commissionAsset': 'TUSD',
   'tradeId': 390514788}],
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [61]:
orders = client.get_all_orders(symbol='BTCTUSD')
orders = pd.DataFrame(orders)
orders

,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,workingTime,origQuoteOrderQty,selfTradePreventionMode
0,BTCTUSD,3642046837,-1,and_84dc04c1d1d84fad8c86a7b5b7f485ed,0.00000000,0.00014000,0.00014000,9.39142960,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1712962352177,1712962352177,True,1712962352177,10.00000000,EXPIRE_MAKER
1,BTCTUSD,3642049567,-1,and_906a064b9618469c8a83f4feb80ae8f9,0.00000000,0.00085000,0.00085000,57.00400190,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1712962538514,1712962538514,True,1712962538514,57.60857040,EXPIRE_MAKER
2,BTCTUSD,3642058873,-1,and_f994cbe299e5442da5c5d06c6fe2986b,0.00000000,0.00099000,0.00099000,66.28255350,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,1712963109701,1712963109701,True,1712963109701,0.00000000,EXPIRE_MAKER
3,BTCTUSD,3642068784,-1,and_48a7991553bf466a8df18138a120796d,0.00000000,0.00099000,0.00099000,66.35086260,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1712963666894,1712963666894,True,1712963666894,66.82415350,EXPIRE_MAKER
4,BTCTUSD,3642082865,-1,and_2bd83a8e16d54525b671d10ce5065507,0.00000000,0.00099000,0.00099000,66.34127610,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,1712964384211,1712964384211,True,1712964384211,0.00000000,EXPIRE_MAKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,BTCTUSD,3642233859,-1,677cfTyvz8cKGyrAmb9Gko,0.00000000,0.00025000,0.00025000,16.64389000,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,1712971381368,1712971381368,True,1712971381368,0.00000000,EXPIRE_MAKER
67,BTCTUSD,3642237349,-1,pvbDXHiqTlqYyaAdQyH6JK,0.00000000,0.00025000,0.00025000,16.64180500,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1712971566716,1712971566716,True,1712971566716,0.00000000,EXPIRE_MAKER
68,BTCTUSD,3642238226,-1,Q5rmwwcuuNdf4A3SZaZ65z,0.00000000,0.00025000,0.00025000,16.63885750,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,1712971607116,1712971607116,True,1712971607116,0.00000000,EXPIRE_MAKER
69,BTCTUSD,3642238968,-1,sVE7LxPYCQKpPjHCHaYGvM,0.00000000,0.00025000,0.00025000,16.64724750,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1712971633765,1712971633765,True,1712971633765,0.00000000,EXPIRE_MAKER


In [86]:
orders['origQty'] = orders['origQty'].astype(float)
orders

,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,workingTime,origQuoteOrderQty,selfTradePreventionMode
0,BTCTUSD,3642046837,-1,and_84dc04c1d1d84fad8c86a7b5b7f485ed,0.00000000,0.00014,0.00014000,9.39142960,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2024-04-12 22:52:32.177,1712962352177,True,1712962352177,10.00000000,EXPIRE_MAKER
1,BTCTUSD,3642049567,-1,and_906a064b9618469c8a83f4feb80ae8f9,0.00000000,0.00085,0.00085000,57.00400190,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2024-04-12 22:55:38.514,1712962538514,True,1712962538514,57.60857040,EXPIRE_MAKER
2,BTCTUSD,3642058873,-1,and_f994cbe299e5442da5c5d06c6fe2986b,0.00000000,0.00099,0.00099000,66.28255350,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,2024-04-12 23:05:09.701,1712963109701,True,1712963109701,0.00000000,EXPIRE_MAKER
3,BTCTUSD,3642068784,-1,and_48a7991553bf466a8df18138a120796d,0.00000000,0.00099,0.00099000,66.35086260,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2024-04-12 23:14:26.894,1712963666894,True,1712963666894,66.82415350,EXPIRE_MAKER
4,BTCTUSD,3642082865,-1,and_2bd83a8e16d54525b671d10ce5065507,0.00000000,0.00099,0.00099000,66.34127610,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,2024-04-12 23:26:24.211,1712964384211,True,1712964384211,0.00000000,EXPIRE_MAKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,BTCTUSD,3642233859,-1,677cfTyvz8cKGyrAmb9Gko,0.00000000,0.00025,0.00025000,16.64389000,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,2024-04-13 01:23:01.368,1712971381368,True,1712971381368,0.00000000,EXPIRE_MAKER
67,BTCTUSD,3642237349,-1,pvbDXHiqTlqYyaAdQyH6JK,0.00000000,0.00025,0.00025000,16.64180500,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2024-04-13 01:26:06.716,1712971566716,True,1712971566716,0.00000000,EXPIRE_MAKER
68,BTCTUSD,3642238226,-1,Q5rmwwcuuNdf4A3SZaZ65z,0.00000000,0.00025,0.00025000,16.63885750,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,2024-04-13 01:26:47.116,1712971607116,True,1712971607116,0.00000000,EXPIRE_MAKER
69,BTCTUSD,3642238968,-1,sVE7LxPYCQKpPjHCHaYGvM,0.00000000,0.00025,0.00025000,16.64724750,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2024-04-13 01:27:13.765,1712971633765,True,1712971633765,0.00000000,EXPIRE_MAKER


In [74]:
orders[orders['origQty'] == 0.00025]

,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,workingTime,origQuoteOrderQty,selfTradePreventionMode
9,BTCTUSD,3642146029,-1,f5jz3LByaEKFEGinynNN96,0.00000000,0.00025,0.00025000,16.75125250,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1712967975319,1712967975319,True,1712967975319,0.00000000,EXPIRE_MAKER
10,BTCTUSD,3642147401,-1,pe44cLiHax8jEmiIYIlzcQ,0.00000000,0.00025,0.00025000,16.75478360,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,1712968062410,1712968062410,True,1712968062410,0.00000000,EXPIRE_MAKER
11,BTCTUSD,3642149592,-1,4RAGhDjHU6vqe8RIIwDaju,0.00000000,0.00025,0.00025000,16.76750000,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1712968183405,1712968183405,True,1712968183405,0.00000000,EXPIRE_MAKER
12,BTCTUSD,3642151905,-1,9ggfYRmim6EDai7xOjKcpf,0.00000000,0.00025,0.00025000,16.77661500,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,1712968230967,1712968230967,True,1712968230967,0.00000000,EXPIRE_MAKER
13,BTCTUSD,3642152708,-1,b0hxiuw8neKoDIDsmqCb9O,0.00000000,0.00025,0.00025000,16.79250000,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1712968251237,1712968251237,True,1712968251237,0.00000000,EXPIRE_MAKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,BTCTUSD,3642233859,-1,677cfTyvz8cKGyrAmb9Gko,0.00000000,0.00025,0.00025000,16.64389000,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,1712971381368,1712971381368,True,1712971381368,0.00000000,EXPIRE_MAKER
67,BTCTUSD,3642237349,-1,pvbDXHiqTlqYyaAdQyH6JK,0.00000000,0.00025,0.00025000,16.64180500,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1712971566716,1712971566716,True,1712971566716,0.00000000,EXPIRE_MAKER
68,BTCTUSD,3642238226,-1,Q5rmwwcuuNdf4A3SZaZ65z,0.00000000,0.00025,0.00025000,16.63885750,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,1712971607116,1712971607116,True,1712971607116,0.00000000,EXPIRE_MAKER
69,BTCTUSD,3642238968,-1,sVE7LxPYCQKpPjHCHaYGvM,0.00000000,0.00025,0.00025000,16.64724750,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1712971633765,1712971633765,True,1712971633765,0.00000000,EXPIRE_MAKER


In [100]:
pmax_df = recent_data.iloc[max_points_idx].loc[:,['Tiempo','Precio']]
pmax_df['Accion'] = 'C'
pmin_df = recent_data.iloc[min_points_idx].loc[:,['Tiempo','Precio']]
pmin_df['Accion'] = 'V'
vc_df = pd.concat([pmax_df,pmin_df],axis=0)
vc_df = vc_df.reset_index()
vc_df = vc_df.set_index('index')
vc_df = vc_df.sort_index()
del_idx = []
# vc_df = vc_df.reset_index()
for i in vc_df.index:
    try:
        if vc_df.Accion[i+1] == vc_df.Accion[i]:
            del_idx.append(i+1)
    except:
        pass
vc_df.drop(del_idx,inplace=True)
vc_df = vc_df.reset_index(drop=True)
vc_df['Diferencia_Precio_USDT'] = pd.Series(dtype='float')
for i in range(len(vc_df)-1):
    vc_df['Diferencia_Precio_USDT'][i+1] = (vc_df.Precio[i+1] - vc_df.Precio[i]) * 0.0001
profits = 0
for i in range(len(vc_df)-1):
    if vc_df.Accion[i] == 'C' and vc_df.Diferencia_Precio_USDT[i+1] > 0:
        profits += vc_df.Diferencia_Precio_USDT[i+1]
    elif vc_df.Accion[i] == 'C' and vc_df.Diferencia_Precio_USDT[i+1] < 0:
        profits -= vc_df.Diferencia_Precio_USDT[i+1]
    elif vc_df.Accion[i] == 'V' and vc_df.Diferencia_Precio_USDT[i+1] > 0:
        profits += vc_df.Diferencia_Precio_USDT[i+1]
    elif vc_df.Accion[i] == 'V' and vc_df.Diferencia_Precio_USDT[i+1] < 0:
        profits -= vc_df.Diferencia_Precio_USDT[i+1]
profits

C:\Users\tinma\AppData\Local\Temp\ipykernel_28576\1265548794.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vc_df['Diferencia_Precio_USDT'][i+1] = (vc_df.Precio[i+1] - vc_df.Precio[i]) * 0.0001


0.39662500000000595

In [99]:
marketorder_sell = client.create_order(
                    symbol='BTCFDUSD',
                    side=Client.SIDE_SELL,
                    type=Client.ORDER_TYPE_MARKET,
                    quantity=trade_qty,
                    recvWindow=60000)
marketorder_sell

{'symbol': 'BTCFDUSD',
 'orderId': 4948003410,
 'orderListId': -1,
 'clientOrderId': 'VLexuKEFbl47cDWBtyzxwf',
 'transactTime': 1712977635537,
 'price': '0.00000000',
 'origQty': '0.00025000',
 'executedQty': '0.00025000',
 'cummulativeQuoteQty': '16.60903250',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'workingTime': 1712977635537,
 'fills': [{'price': '66436.13000000',
   'qty': '0.00025000',
   'commission': '0.00000000',
   'commissionAsset': 'BNB',
   'tradeId': 592697119}],
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [ ]:
client.create_order(
                    symbol='BTCUSDT',
                    side=Client.SIDE_SELL,
                    type=Client.ORDER_TYPE_MARKET,
                    quantity=trade_qty,
                    recvWindow=60000)

In [ ]:
client.create_order(
                    symbol='BTCUSDT',
                    side=Client.SIDE_BUY,
                    type=Client.ORDER_TYPE_MARKET,
                    quantity=trade_qty,
                    recvWindow=60000)

In [15]:
client.get_account(recvWindow=50000) # BEFORE doing the strategy again: USDT: 9924.14895610, BTC: 1.00100000

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1712728469072,
 'accountType': 'SPOT',
 'balances': [{'asset': 'ETH', 'free': '0.99700000', 'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00100000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '5.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '9924.12948920', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '3798.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '737.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '27.00000000', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '91.00000000

In [11]:
sell_orders

[{'symbol': 'BTCUSDT',
  'orderId': 15932319,
  'orderListId': -1,
  'clientOrderId': 'loOzpecAAwKs70KIzsayar',
  'transactTime': 1712727399647,
  'price': '0.00000000',
  'origQty': '0.00100000',
  'executedQty': '0.00100000',
  'cummulativeQuoteQty': '69.19547000',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'SELL',
  'workingTime': 1712727399647,
  'fills': [{'price': '69195.47000000',
    'qty': '0.00100000',
    'commission': '0.00000000',
    'commissionAsset': 'USDT',
    'tradeId': 3692926}],
  'selfTradePreventionMode': 'EXPIRE_MAKER'},
 {'symbol': 'BTCUSDT',
  'orderId': 15932394,
  'orderListId': -1,
  'clientOrderId': 'Sm5TurYieLBVQaixYTxsvf',
  'transactTime': 1712727419800,
  'price': '0.00000000',
  'origQty': '0.00100000',
  'executedQty': '0.00100000',
  'cummulativeQuoteQty': '69.19547000',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'SELL',
  'workingTime': 1712727419800,
  'fills': [{'price': '69195.470

In [12]:
buy_orders

[{'symbol': 'BTCUSDT',
  'orderId': 15932275,
  'orderListId': -1,
  'clientOrderId': '41RRw2jvJtRsEcgRJAgXwI',
  'transactTime': 1712727387516,
  'price': '0.00000000',
  'origQty': '0.00100000',
  'executedQty': '0.00100000',
  'cummulativeQuoteQty': '69.36137000',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'BUY',
  'workingTime': 1712727387516,
  'fills': [{'price': '69361.37000000',
    'qty': '0.00100000',
    'commission': '0.00000000',
    'commissionAsset': 'BTC',
    'tradeId': 3692900}],
  'selfTradePreventionMode': 'EXPIRE_MAKER'},
 {'symbol': 'BTCUSDT',
  'orderId': 15932350,
  'orderListId': -1,
  'clientOrderId': 'Tueo5OFE1R2dhAPBGXbetl',
  'transactTime': 1712727408748,
  'price': '0.00000000',
  'origQty': '0.00100000',
  'executedQty': '0.00100000',
  'cummulativeQuoteQty': '69.36177000',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'BUY',
  'workingTime': 1712727408748,
  'fills': [{'price': '69361.770000

In [14]:
pmax_df = recent_data.iloc[max_points_idx].loc[:,['Tiempo','Precio']]
pmax_df['Accion'] = 'C'
pmin_df = recent_data.iloc[min_points_idx].loc[:,['Tiempo','Precio']]
pmin_df['Accion'] = 'V'
vc_df = pd.concat([pmax_df,pmin_df],axis=0)
vc_df = vc_df.reset_index()
vc_df = vc_df.set_index('index')
vc_df = vc_df.sort_index()
del_idx = []
# vc_df = vc_df.reset_index()
for i in vc_df.index:
    try:
        if vc_df.Accion[i+1] == vc_df.Accion[i]:
            del_idx.append(i+1)
    except:
        pass
vc_df.drop(del_idx,inplace=True)
vc_df = vc_df.reset_index(drop=True)
vc_df['Diferencia_Precio_USDT'] = pd.Series(dtype='float')
for i in range(len(vc_df)-1):
    vc_df['Diferencia_Precio_USDT'][i+1] = (vc_df.Precio[i+1] - vc_df.Precio[i]) * 0.001
profits = 0
for i in range(len(vc_df)-1):
    if vc_df.Accion[i] == 'C' and vc_df.Diferencia_Precio_USDT[i+1] > 0:
        profits += vc_df.Diferencia_Precio_USDT[i+1]
    elif vc_df.Accion[i] == 'C' and vc_df.Diferencia_Precio_USDT[i+1] < 0:
        profits -= vc_df.Diferencia_Precio_USDT[i+1]
    elif vc_df.Accion[i] == 'V' and vc_df.Diferencia_Precio_USDT[i+1] > 0:
        profits += vc_df.Diferencia_Precio_USDT[i+1]
    elif vc_df.Accion[i] == 'V' and vc_df.Diferencia_Precio_USDT[i+1] < 0:
        profits -= vc_df.Diferencia_Precio_USDT[i+1]
profits

C:\Users\tinma\AppData\Local\Temp\ipykernel_31372\2802205712.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vc_df['Diferencia_Precio_USDT'][i+1] = (vc_df.Precio[i+1] - vc_df.Precio[i]) * 0.001


0.5711799999999637

In [33]:
# recent_data = pd.read_csv('../../../csvs/streaming/BTCUSDT.csv')
# recent_data.columns = ['Tiempo','Simbolo','Precio']
# recent_data = recent_data.loc[:,['Tiempo','Precio']]
recent_data

,Tiempo,Precio


In [25]:
500 % 500 == 0

True

In [33]:
marketorder_buy = client.create_order(
                        symbol='XRPETH',
                        side=Client.SIDE_BUY,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=0.003,
                        recvWindow=10000)
marketorder_buy

# marketorder_buy ----> Diccionario: symbol, orderId, transactTime, orig_Qty, side, workingTime, fills ----> Lista con diccionarios. Keys: price, qty, comission, comissionaAsset, tradeId

{'symbol': 'XRPUSDT',
 'orderId': 4156737,
 'orderListId': -1,
 'clientOrderId': 'o4H8gQaUWCFIfR0cZgZlK0',
 'transactTime': 1712666256119,
 'price': '0.00000000',
 'origQty': '10.00000000',
 'executedQty': '10.00000000',
 'cummulativeQuoteQty': '6.36400000',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'BUY',
 'workingTime': 1712666256119,
 'fills': [{'price': '0.63640000',
   'qty': '10.00000000',
   'commission': '0.00000000',
   'commissionAsset': 'XRP',
   'tradeId': 266819}],
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [26]:
marketorder_sell = client.create_order(
                        symbol='ETHUSDT',
                        side=Client.SIDE_SELL,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=0.003,
                        recvWindow=10000)
marketorder_sell

{'symbol': 'ETHUSDT',
 'orderId': 8795137,
 'orderListId': -1,
 'clientOrderId': 'mGuWjb2npgk5KI8u2UdGVQ',
 'transactTime': 1712665283539,
 'price': '0.00000000',
 'origQty': '0.00300000',
 'executedQty': '0.00300000',
 'cummulativeQuoteQty': '10.88259000',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'workingTime': 1712665283539,
 'fills': [{'price': '3627.53000000',
   'qty': '0.00300000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 1818593}],
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [27]:
client.get_account(recvWindow=10000)

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1712665283539,
 'accountType': 'SPOT',
 'balances': [{'asset': 'ETH', 'free': '0.99700000', 'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '5.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '10000.21963610', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '3798.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '727.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '27.00000000', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '91.0000000

In [ ]:
# balances = []
# ganancia_maxima = 0
# open_order = False
# balance_trade_anterior = 0
# balance_trade_p = 0
# tiempo_inicial = datetime.utcnow()
# #### PARÁMETROS DE LAS CONDICIONES DE SALIDA Y ENTRADA.
# PAIR = 'BTCUSDT'
# pair = 'btc'
# LOCALIDAD_SEÑAL_ENTRADA = 30 # EN SEGUNDOS
# # El primero de ellos es el promedio de la pendiente (precio/tiempo) de las últimas 10 filas del dataframe. Se utiliza para 
# # saber si se debe entrar en una posición corta al alza (puesto que a la baja no estamos tradeando) junto con otra condición
# PENDIENTE_ENTRADA = 0.02
# # LOS UMBRALES SON EN DÓLARES.
# UMBRAL_PERDIDA_1P = - 5
# UMBRAL_GANANCIA_1G = 20
# UMBRAL_GANANCIA_3G = 35
# UMBRAL_GANANCIA_4G = 5
# UMBRAL_MAXIMO_2G = 20
# UMBRAL_MINIMO_2G = 5
# LOCALIDAD_SEÑAL_SALIDA_2G = 30 # EN SEGUNDOS
# LIMITE_TIEMPO_POSITIVO_4G = 10 # EN MINUTOS
# log_path = f'../LOGS/{estrategia}.csv'
# df_logs = pd.DataFrame(columns=['Tiempo','Accion'])




# ##############  VERIFICAMOS SI NO HAY OPERACIONES ABIERTAS: Como solo vamos a trabajar con señales de entrada, toda posición
# ##############  está cerrada si la siguiente condición se cumple.

#     if open_order == False:

# ##############  EN CASO DE NO HABER, SE PROCEDE A VERIFICAR LA "LOCALIDAD" DE LA SEÑAL: En caso de ser una señal local se crea 
# ##############  una orden de compra y se inicia el proceso.

#         # pendientes = [0]

#         # for i in range(len(df)-1):
#         #     x = slopee(df.index[i],df.Precio[i],df.index[i+1],df.Precio[i+1])
#         #     pendientes.append(x)

#         # df['pendiente'] = pendientes

#         DIFF_SEÑAL_ENTRADA = (datetime.utcnow() - TIEMPO_SEÑAL_ENTRADA).total_seconds()

#         if DIFF_SEÑAL_ENTRADA < LOCALIDAD_SEÑAL_ENTRADA: # and df.pendiente[-10:-1].mean() > PENDIENTE_ENTRADA:
#             buy_order = client.create_order(
#                         symbol=PAIR,
#                         side=Client.SIDE_BUY,
#                         type=Client.ORDER_TYPE_MARKET,
#                         quantity=cantidad_trade,
#                         recvWindow=60000
#                     )
#             open_order = True
#             tiempo_entrada = datetime.utcnow()
#             precio_entrada = float(buy_order['fills'][0]['price'])
#             print(f'Se realizó una compra al precio: {precio_entrada}, en el momento: {tiempo_entrada}.')

# ##############  SI EXISTE UNA OPERACIÓN ABIERTA, LA SIGUIENTE CONDICIÓN SE CUMPLE: En ese caso, se debe estar atento a cuándo
# ##############  cerrar nuestra posición. Ya sea que estemos ganando o perdiendo; se establecen límites de pérdida y de ganancia,
# ##############  así como también casos en los que las señales de venta se toman en cuenta, tanto en pérdidas como en ganancias.

#     while open_order:

#         day = datetime.utcnow() - timedelta(seconds=1000)

#         with connection.connect() as conn:
#             df =conn.execute(text(""" SELECT * FROM "{PAIR}" WHERE "Tiempo" > '{day}' """.format(day=day,PAIR=PAIR))).mappings().all()
#         df = pd.DataFrame(df)[['Tiempo', 'Precio']].set_index('Tiempo')
#         df = df.sort_values(by='Tiempo')
#         MACD(df)
#         buy, sell = señal_MACD(df)

#         buyprices = df.Precio.iloc[buy]
#         sellprices = df.Precio.iloc[sell]
#         TIEMPO_SEÑAL_SALIDA = sellprices.index[-1]
#         precio_actual = df.Precio[-1]

#         profit = (precio_actual - precio_entrada)

#         if profit == 0:
#             continue 

# ##############  SEÑALES DE VENTA EN CASO DE TENER PÉRDIDAS.

# #############  La primera señal se enfoca en no superar el umbral de pérdidas.       

#         elif profit < 0:

#             if open_order:

#                 if profit < UMBRAL_PERDIDA_1P:
#                     sell_order = client.create_order(
#                                 symbol=PAIR,
#                                 side=Client.SIDE_SELL,
#                                 type=Client.ORDER_TYPE_MARKET,
#                                 quantity=cantidad_trade,
#                                 recvWindow=60000
#                             )
#                     open_order = False
#                     Metodo = '1P'
#                     tiempo_salida = datetime.utcnow()
#                     precio_salida = float(sell_order['fills'][0]['price'])
#                     balance_trade_p = (precio_salida - precio_entrada)*cantidad_trade
#                     balances.append(balance_trade_p)
                        
# ##############  SEÑALES DE VENTA EN CASO DE TENER GANANCIAS.

#         else:

# #############  La primera señal se enfoca en no perder todas las ganancias, en caso de existir ganancias. Se permite una pérdida
# #############  del 50% de la ganancia máxima.


#             if open_order:

#                 if profit > UMBRAL_GANANCIA_1G:

#                     if profit >= ganancia_maxima:
#                         ganancia_maxima = profit
#                     elif profit < ganancia_maxima:
#                         if (profit/ganancia_maxima - 1) < - 0.2: 
#                             sell_order = client.create_order(
#                                         symbol=PAIR,
#                                         side=Client.SIDE_SELL,
#                                         type=Client.ORDER_TYPE_MARKET,
#                                         quantity=cantidad_trade,
#                                         recvWindow=60000
#                                     )
#                             open_order = False
#                             Metodo = '1G'
#                             tiempo_salida = datetime.utcnow()
#                             precio_salida = float(sell_order['fills'][0]['price'])
#                             balance_trade_p = (precio_salida - precio_entrada)*cantidad_trade
#                             balances.append(balance_trade_p)

# #############  La segunda señal se enfoca en vender en caso de alcanzar ganancias mínimas del 0.07% y a la vez se da una señal
# #############  de venta.   

#             if open_order:     

#                 if profit <= UMBRAL_MAXIMO_2G and profit > UMBRAL_MINIMO_2G and (datetime.utcnow() - TIEMPO_SEÑAL_SALIDA).total_seconds() < LOCALIDAD_SEÑAL_SALIDA_2G:
#                     sell_order = client.create_order(
#                                 symbol=PAIR,
#                                 side=Client.SIDE_SELL,
#                                 type=Client.ORDER_TYPE_MARKET,
#                                 quantity=cantidad_trade,
#                                 recvWindow=60000
#                             )
#                     open_order = False
#                     Metodo = '2G'
#                     tiempo_salida = datetime.utcnow()
#                     precio_salida = float(sell_order['fills'][0]['price'])
#                     balance_trade_p = (precio_salida - precio_entrada)*cantidad_trade
#                     balances.append(balance_trade_p)
                    
# #############  La tercer señal se enfoca en vender en caso de alcanzar ganancias máximas del 1.4%

#             if open_order:
            
#                 if profit > UMBRAL_GANANCIA_3G:
#                     sell_order = client.create_order(
#                                 symbol=PAIR,
#                                 side=Client.SIDE_SELL,
#                                 type=Client.ORDER_TYPE_MARKET,
#                                 quantity=cantidad_trade,
#                                 recvWindow=60000
#                             )
#                     open_order = False
#                     Metodo = '3G'
#                     tiempo_salida = datetime.utcnow()
#                     precio_salida = float(sell_order['fills'][0]['price'])
#                     balance_trade_p = (precio_salida - precio_entrada)*cantidad_trade
#                     balances.append(balance_trade_p)

# ############# POR ÚLTIMO, VAMOS A GENERAR UNA VENTA SI EL TRADE ESTÁ EN GANANCIAS PERO NO SE HA VENDIDO DESPUÉS DE 10 MINUTOS.

#             if open_order:
            
#                 if ((datetime.utcnow() - tiempo_entrada).total_seconds()/60) > LIMITE_TIEMPO_POSITIVO_4G and profit > UMBRAL_GANANCIA_4G:
#                     sell_order = client.create_order(
#                                 symbol=PAIR,
#                                 side=Client.SIDE_SELL,
#                                 type=Client.ORDER_TYPE_MARKET,
#                                 quantity=cantidad_trade,
#                                 recvWindow=60000
#                             )
#                     open_order = False
#                     Metodo = '4G'
#                     tiempo_salida = datetime.utcnow()
#                     precio_salida = float(sell_order['fills'][0]['price'])
#                     balance_trade_p = (precio_salida - precio_entrada)*cantidad_trade
#                     balances.append(balance_trade_p)

######################## LOGS

    # if balance_trade_anterior != balance_trade_p:
    #     balance_trade_anterior = balance_trade_p
    #     if balance_trade_p>0:
    #         print(f'Se realizó una venta al precio: {precio_salida}, en el momento: {tiempo_salida}. Ganancia de: {balance_trade_p} dólares. \nMétodo: {Metodo}')
    #         trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance_trade_p),Metodo,LOCALIDAD_SEÑAL_ENTRADA,PENDIENTE_ENTRADA,UMBRAL_PERDIDA_1P,UMBRAL_GANANCIA_1G,UMBRAL_GANANCIA_3G,UMBRAL_GANANCIA_4G,UMBRAL_MAXIMO_2G,UMBRAL_MINIMO_2G,LOCALIDAD_SEÑAL_SALIDA_2G,LIMITE_TIEMPO_POSITIVO_4G,DIFF_SEÑAL_ENTRADA]],
    #                                     columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Ganancia','Método','param_1','param_2','param_3','param_4','param_5','param_6','param_7','param_8','param_9','param_10','param_11'])
    #         if os.path.exists(path+f'trades_ganados_{pair}.csv'):
    #             trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
    #                             mode='a',
    #                             index=False,
    #                             header=False
    #                             )
    #         else:
    #             trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
    #                             index=False,
    #                             header=True,
    #                             )
    #     elif balance_trade_p==0:
    #         print('No hubo pérdidas ni ganancias del trade')
    #     else:
    #         print(f'Se realizó una venta al precio: {precio_salida}, en el momento: {tiempo_salida}. Pérdida de: {balance_trade_p} dólares. \nMétodo: {Metodo}')
    #         trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance_trade_p),Metodo,LOCALIDAD_SEÑAL_ENTRADA,PENDIENTE_ENTRADA,UMBRAL_PERDIDA_1P,UMBRAL_GANANCIA_1G,UMBRAL_GANANCIA_3G,UMBRAL_GANANCIA_4G,UMBRAL_MAXIMO_2G,UMBRAL_MINIMO_2G,LOCALIDAD_SEÑAL_SALIDA_2G,LIMITE_TIEMPO_POSITIVO_4G,DIFF_SEÑAL_ENTRADA]],
    #                                     columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Perdida','Método','param_1','param_2','param_3','param_4','param_5','param_6','param_7','param_8','param_9','param_10','param_11'])
    #         if os.path.exists(path+f'trades_perdidos_{pair}.csv'):
    #             trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
    #                             mode='a',
    #                             index=False,
    #                             header=False
    #                             )
    #         else:
    #             trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
    #                             index=False,
    #                             header=True,
    #                             )

In [3]:
# path = 'C:/Users/tinma/OneDrive/Escritorio/tradingbot/trade_logs/testnet/logs/Sesiones/MACD_MACD+logicv1/'

# # sleep(5)
    
# if open_order:
#     sell_order = client.create_order(
#                             symbol=PAIR,
#                             side=Client.SIDE_SELL,
#                             type=Client.ORDER_TYPE_MARKET,
#                             quantity=cantidad_trade
#                         )
#     open_order = False
#     precio_salida = float(sell_order['fills'][0]['price'])
#     balance_trade_p = (precio_salida - precio_entrada)*cantidad_trade
#     balances.append(balance_trade_p)
    
# if tiempo_entrada > tiempo_salida:
#     tiempo_final = tiempo_entrada
# elif tiempo_entrada < tiempo_salida:
#     tiempo_final = tiempo_salida 

# if sum(balances) > 0:
#     session_log = pd.DataFrame([[len(balances), sum(balances), sum(balances)/len(balances), PAIR, estrategia, tiempo_inicial, tiempo_final,cantidad_trade]],
#              columns=['Cantidad_Trades','Ganancia','Ganancia_Promedio_xTrade','PAR','Estrategia','Tiempo_inicial','Tiempo_final','Cantidad_trade'])
#     if os.path.exists(path+'Sesiones_Ganadas.csv'):
#         session_log.to_csv(path+'Sesiones_Ganadas.csv',
#                         mode='a',
#                         index=False,
#                         header=False
#                         )
#     else:
#         session_log.to_csv(path+'Sesiones_Ganadas.csv',
#                         index=False,
#                         header=True
#                         )
# elif sum(balances) < 0:
#     session_log = pd.DataFrame([[len(balances), sum(balances), sum(balances)/len(balances), PAIR, estrategia, tiempo_inicial, tiempo_final,cantidad_trade]],
#              columns=['Cantidad_Trades','Perdida','Ganancia_Promedio_xTrade','PAR','Estrategia','Tiempo_inicial','Tiempo_final','Cantidad_trade'])
#     if os.path.exists(path+'Sesiones_Perdidas.csv'):
#         session_log.to_csv(path+'Sesiones_Perdidas.csv',
#                         mode='a',
#                         index=False,
#                         header=False
#                         )
#     else:
#         session_log.to_csv(path+'Sesiones_Perdidas.csv',
#                         index=False,
#                         header=True
#                         )

In [4]:
session_log

,Cantidad_Trades,Ganancia,Ganancia_Promedio_xTrade,PAR,Estrategia,Tiempo_inicial,Tiempo_final,Cantidad_trade
0,44,0.14105,0.003206,BTCUSDT,MACD_logicv1,2023-07-05 13:55:44.163789,2023-07-05 14:33:14.443281,0.005


In [12]:
# balances[-1] = balances[-1]*0.1
balances

[0.08279999999998836,
 0.07739999999997962,
 0.05650000000001455,
 -0.05930000000000291,
 0.057000000000007274,
 0.20440000000002329]

In [28]:
sum(balances) + 1.5547999999999957

-0.09275000000008782

In [4]:
client.get_account()

BinanceAPIException: APIError(code=-1021): Timestamp for this request is outside of the recvWindow.

In [38]:
sum(balances)

-55.60899999998987

In [117]:
pd.to_datetime('2023-06-29 20:03:37.435685') - pd.to_datetime('2023-06-29 19:05:24.924680')

Timedelta('0 days 00:58:12.511005')

In [121]:
balances[-1]

-0.7869999999998982

In [39]:

buy_order = client.create_order(
                        symbol='BTCUSDT',
                        side=Client.SIDE_BUY,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=0.1
                    )

In [32]:
sell_order = client.create_order(
                            symbol='BTCUSDT',
                            side=Client.SIDE_SELL,
                            type=Client.ORDER_TYPE_MARKET,
                            quantity=0.1
                        )

In [16]:
sell_order

{'symbol': 'BTCUSDT',
 'orderId': 10464206,
 'orderListId': -1,
 'clientOrderId': 'zmNXnF77mLQvDiTWZ94hZS',
 'transactTime': 1688165636752,
 'price': '0.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '3046.84891893',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'workingTime': 1688165636752,
 'fills': [{'price': '30468.63000000',
   'qty': '0.00859600',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2990498},
  {'price': '30468.55000000',
   'qty': '0.06432900',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2990499},
  {'price': '30468.30000000',
   'qty': '0.02707500',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2990500}],
 'selfTradePreventionMode': 'NONE'}

In [5]:
marketorder_buy = client.create_order(
    symbol='BTCUSDT',
    side=Client.SIDE_BUY,
    type=Client.ORDER_TYPE_MARKET,
    quantity=0.1,
    recvWindow=10000)

In [16]:
marketorder_sell = client.create_order(
    symbol='BTCUSDT',
    side=Client.SIDE_SELL,
    type=Client.ORDER_TYPE_MARKET,
    quantity=0.1,
    recvWindow=10000)

In [17]:
limitorder_sell

{'symbol': 'BTCUSDT',
 'orderId': 15582800,
 'orderListId': -1,
 'clientOrderId': 'UbJC3pf8YldNgFrnFPYmys',
 'transactTime': 1712660072002,
 'price': '0.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '7057.64658920',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'workingTime': 1712660072002,
 'fills': [{'price': '70620.43000000',
   'qty': '0.00574000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 3611441},
  {'price': '70620.30000000',
   'qty': '0.00390000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 3611442},
  {'price': '70618.14000000',
   'qty': '0.00100000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 3611443},
  {'price': '70614.61000000',
   'qty': '0.00100000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 3611444},
  {'price': '70613.49000000',
   'qty': '0.00393000',
   'commi

In [18]:
client.get_all_orders(symbol='BTCUSDT',recvWindow=10000) 15609234

order = client.get_order(
    symbol='BTCUSDT',
    orderId='15582800',
    recvWindow=10000)

In [19]:
order

{'symbol': 'BTCUSDT',
 'orderId': 15582800,
 'orderListId': -1,
 'clientOrderId': 'UbJC3pf8YldNgFrnFPYmys',
 'price': '0.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '7057.64658920',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'stopPrice': '0.00000000',
 'icebergQty': '0.00000000',
 'time': 1712660072002,
 'updateTime': 1712660072002,
 'isWorking': True,
 'workingTime': 1712660072002,
 'origQuoteOrderQty': '0.00000000',
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [10]:
cancel_order = client.cancel_order(
    symbol='BTCUSDT',
    orderId='10459808')

BinanceAPIException: APIError(code=-2011): Unknown order sent.

In [15]:
limitorder

{'symbol': 'BTCUSDT',
 'orderId': 10459808,
 'orderListId': -1,
 'clientOrderId': 'Vgl1y7Y4unFF40r881ULls',
 'transactTime': 1688164704216,
 'price': '30800.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '3051.23838466',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'BUY',
 'workingTime': 1688164704216,
 'fills': [{'price': '30512.38000000',
   'qty': '0.08717800',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 2989981},
  {'price': '30512.41000000',
   'qty': '0.01282200',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 2989982}],
 'selfTradePreventionMode': 'NONE'}

In [92]:
sell_order

{'symbol': 'BTCUSDT',
 'orderId': 9914874,
 'orderListId': -1,
 'clientOrderId': 'n3IbYFnRv5COlykPamCSvW',
 'transactTime': 1688061602684,
 'price': '0.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '3054.32647239',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'workingTime': 1688061602684,
 'fills': [{'price': '30543.33000000',
   'qty': '0.02747100',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2793732},
  {'price': '30543.24000000',
   'qty': '0.05140300',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2793733},
  {'price': '30543.24000000',
   'qty': '0.02112600',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2793734}],
 'selfTradePreventionMode': 'NONE'}

In [20]:
client.get_account(recvWindow=10000)

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1712660072002,
 'accountType': 'SPOT',
 'balances': [{'asset': 'ETH', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '0.90000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '5.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '17057.64658920', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '3798.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '727.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '27.00000000', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '91.0000000

In [34]:
client.get_all_orders(symbol='BTCUSDT',recvWindow=10000) 15609234

[{'symbol': 'BTCUSDT',
  'orderId': 15582800,
  'orderListId': -1,
  'clientOrderId': 'UbJC3pf8YldNgFrnFPYmys',
  'price': '0.00000000',
  'origQty': '0.10000000',
  'executedQty': '0.10000000',
  'cummulativeQuoteQty': '7057.64658920',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'SELL',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1712660072002,
  'updateTime': 1712660072002,
  'isWorking': True,
  'workingTime': 1712660072002,
  'origQuoteOrderQty': '0.00000000',
  'selfTradePreventionMode': 'EXPIRE_MAKER'},
 {'symbol': 'BTCUSDT',
  'orderId': 15609234,
  'orderListId': -1,
  'clientOrderId': 'E9ilsoPQPBOt3ypxACHUE7',
  'price': '0.00000000',
  'origQty': '0.10000000',
  'executedQty': '0.10000000',
  'cummulativeQuoteQty': '7068.30954310',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'BUY',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1712663642250,
  'updateTime': 17126636